## End-to-End Job Market Analytics ETL Pipeline using Databricks & SerpAPI

### Project Description

This project demonstrates an end-to-end Job Aggregator ETL pipeline built using SerpAPI (Google Jobs API), Apache Spark, and Databricks to collect, process, and analyze software and IT job listings.

The pipeline follows the Medallion Architecture to ensure scalable, reliable, and analytics-ready data processing:

🥉 Bronze Layer: Ingests raw job listings from SerpAPI and stores them in Delta tables with schema evolution support.

🥈 Silver Layer: Cleans and standardizes job data by handling null values, removing duplicates, and normalizing text fields to create high-quality datasets.

🥇 Gold Layer: Generates aggregated, business-ready tables to answer analytical questions such as top hiring roles, companies, and locations.

### 🥉 BRONZE LAYER: Ingesting Data to Databricks

Fetching Software Job Listings using SerpAPI (Google Jobs API)

In [0]:
#Import Required Libraries
import requests
import json 
from pyspark.sql import SparkSession

#Create Spark Session
spark = SparkSession.builder.appName("JobETL_Bronze").getOrCreate()

#Define API & Search Parameters
API_KEY = "b1a838a281e3ca55a62d29141e876ac137c83404c2f17dc4abdb0929a9ff9839"

# Broad IT / Software job search (VERY IMPORTANT)
roles = [
    "Software Engineer",
    "IT Jobs",
    "Technology Jobs",
    "Backend Developer",    
    "Full Stack Developer",
    "Data Engineer",
    "DevOps Engineer",
    "Frontend Developer",
    "Data Scientist"
]

location = "India"

# Fetch Jobs from SerpApi (WITH PAGINATION)
all_jobs = []
print("Starting Job Fetching....\n")

for role in roles:
    print(f"Fetching jobs for role: {role}")

    next_page_token = None
    page_count = 1   # safety counter

    while True:
        print(f"  Fetching page {page_count}")

        params = {
            "engine": "google_jobs",
            "q": role,
            "location": location,
            "api_key": API_KEY
        }

        if next_page_token:
            params["next_page_token"] = next_page_token

        response = requests.get("https://serpapi.com/search.json", params=params)
        data = response.json()

        jobs = data.get("jobs_results", [])

        if not jobs:
            break

        for job in jobs:
            job["search_role"] = role
            simplified_job = {
                "title": job.get("title"),
                "company_name": job.get("company_name"),
                "location": job.get("location"),
                "via": job.get("via"),
                "share_link": job.get("share_link"),
                "search_role": role
            }
            all_jobs.append(simplified_job)

        #all_jobs.extend(jobs)

        next_page_token = data.get("next_page_token")

        if not next_page_token:
            break

        page_count += 1

print("\nJob Fetching Completed")
print(f"Total Jobs Collected: {len(all_jobs)}")


Starting Job Fetching....

Fetching jobs for role: Software Engineer
  Fetching page 1
Fetching jobs for role: IT Jobs
  Fetching page 1
Fetching jobs for role: Technology Jobs
  Fetching page 1
Fetching jobs for role: Backend Developer
  Fetching page 1
Fetching jobs for role: Full Stack Developer
  Fetching page 1
Fetching jobs for role: Data Engineer
  Fetching page 1
Fetching jobs for role: DevOps Engineer
  Fetching page 1
Fetching jobs for role: Frontend Developer
  Fetching page 1
Fetching jobs for role: Data Scientist
  Fetching page 1

Job Fetching Completed
Total Jobs Collected: 90


Inspecting Fetched Job Data

- View the keys of the first job record

- Display a few sample jobs

In [0]:
#Viewing the structure of the first job record and a few sample jobs
print(all_jobs[0].keys())
display(all_jobs[:3])


dict_keys(['title', 'company_name', 'location', 'via', 'share_link', 'search_role'])


company_name,location,search_role,share_link,title,via
ServiceNow,"Hyderabad, Telangana, India",Software Engineer,https://www.google.com/search?ibp=htl;jobs&q=Software+Engineer&htidocid=PCIMKdItwxs7RmWhAAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xWNsQ6CMBBA48onON1kokEwJi66uBjUwZjgTq7laGvwjrSNwC_5lcLyhveGl_wWybmM2DRQShN79AQXNo6JPGzhOUYrDCsopEU2k7mLgkDotYUpFCKmpeXJxtiFY56H0GYmRIxOZ1o-uTApGfK3qDCjCnYadC1GqvaH3ZB1bDbrkvzXaXpID47hOtbkUWGdwovmKTKmcOPa4R-diUZBrQAAAA&shmds=v1_ATWGeePv2NJedJdFralF3FTkQ0wpIfl1KYUiOmnSHwdY1DIEZA&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=Software+Engineer&htidocid=PCIMKdItwxs7RmWhAAAAAA%3D%3D,Staff Software Engineer - Python & Golang,ServiceNow Careers
JPMC Candidate Experience page,"Hyderabad, Telangana, India",Software Engineer,https://www.google.com/search?ibp=htl;jobs&q=Software+Engineer&htidocid=C8-O3Oo_8QlZzDQ7AAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xWNQQrCMBBFcdsjuJq11EYENxVXpWgLgqj7Mk3HNKXOhCRovZnHs24-PB6Pn3wXSX6TR3yjJyjZWCbyUFWwhiuhjkOAGl84Yy0tBEKvexCGo4gZabnvY3QhVyqEMTMhYrQ60_JUwtTKpAZpw3-a0M8HbsRIzXa3mTLHZnWoL-cCCuTOdrOAcnLkLbEmcGgILMPp05HHFrsU7jQiG2RMoZoL_AGCpoXmuwAAAA&shmds=v1_ATWGeeNJb8XUgXPtJlkeDlbrA73EJm5eYfDabQBHJdJ5sHk0Bw&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=Software+Engineer&htidocid=C8-O3Oo_8QlZzDQ7AAAAAA%3D%3D,Software Engineer II - Reactjs Java,JPMorgan Chase Login
GE Vernova,Anywhere,Software Engineer,https://www.google.com/search?ibp=htl;jobs&q=Software+Engineer&htidocid=7VdAOS-XTTlXUQMAAAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xXMMQ7CMAwAQLH2CUxeQaUBJBaYqwIriLVyGpMGpXYVR1DewmeB5cYrPrNiUw-WnCMHF7nnFyaCmn1gogQrOIsFJUxdD8LQiPhI80Of86h7Y1Rj5TVjDl3VyWCEycpkHmL1T6v9rxsjZmq3u_VUjeyXi6aGGyWWJ0JgOL4dJbToSrhSRPbIWMKJXcAvsJ1QkZ4AAAA&shmds=v1_ATWGeeMu-oeyp3h7he1x7KcvL61yNFxkLmLpeEBgihl62kqgBw&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=Software+Engineer&htidocid=7VdAOS-XTTlXUQMAAAAAAA%3D%3D,Embedded Software Engineer,GE Vernova Careers


View RAW JSON

- Wrap the fetched jobs in a JSON-like dictionary

- Display total jobs and sample data

In [0]:
#View RAW JSON
job_data = {
    "total_jobs": len(all_jobs),
    "jobs": all_jobs
}

display(job_data)


{'total_jobs': 90,
 'jobs': [{'title': 'Staff Software Engineer - Python & Golang',
   'company_name': 'ServiceNow',
   'location': 'Hyderabad, Telangana, India',
   'via': 'ServiceNow Careers',
   'share_link': 'https://www.google.com/search?ibp=htl;jobs&q=Software+Engineer&htidocid=PCIMKdItwxs7RmWhAAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xWNsQ6CMBBA48onON1kokEwJi66uBjUwZjgTq7laGvwjrSNwC_5lcLyhveGl_wWybmM2DRQShN79AQXNo6JPGzhOUYrDCsopEU2k7mLgkDotYUpFCKmpeXJxtiFY56H0GYmRIxOZ1o-uTApGfK3qDCjCnYadC1GqvaH3ZB1bDbrkvzXaXpID47hOtbkUWGdwovmKTKmcOPa4R-diUZBrQAAAA&shmds=v1_ATWGeePv2NJedJdFralF3FTkQ0wpIfl1KYUiOmnSHwdY1DIEZA&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=Software+Engineer&htidocid=PCIMKdItwxs7RmWhAAAAAA%3D%3D',
   'search_role': 'Software Engineer'},
  {'title': 'Software Engineer II - Reactjs Java',
   'company_name': 'JPMC Candidate Experience page',
   'location': 'Hyderabad, Telangana, India',
   'via': 'JPMorgan Chase Login',
   'share_link': 'https://www.go

Convert to Spark DataFrame

- Convert the Python list of dictionaries to a Spark DataFrame

- Display the DataFrame

In [0]:
#Convert to Spark DataFrame
bronze_df = spark.createDataFrame(all_jobs)
bronze_df.display()


company_name,location,search_role,share_link,title,via
ServiceNow,"Hyderabad, Telangana, India",Software Engineer,https://www.google.com/search?ibp=htl;jobs&q=Software+Engineer&htidocid=PCIMKdItwxs7RmWhAAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xWNsQ6CMBBA48onON1kokEwJi66uBjUwZjgTq7laGvwjrSNwC_5lcLyhveGl_wWybmM2DRQShN79AQXNo6JPGzhOUYrDCsopEU2k7mLgkDotYUpFCKmpeXJxtiFY56H0GYmRIxOZ1o-uTApGfK3qDCjCnYadC1GqvaH3ZB1bDbrkvzXaXpID47hOtbkUWGdwovmKTKmcOPa4R-diUZBrQAAAA&shmds=v1_ATWGeePv2NJedJdFralF3FTkQ0wpIfl1KYUiOmnSHwdY1DIEZA&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=Software+Engineer&htidocid=PCIMKdItwxs7RmWhAAAAAA%3D%3D,Staff Software Engineer - Python & Golang,ServiceNow Careers
JPMC Candidate Experience page,"Hyderabad, Telangana, India",Software Engineer,https://www.google.com/search?ibp=htl;jobs&q=Software+Engineer&htidocid=C8-O3Oo_8QlZzDQ7AAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xWNQQrCMBBFcdsjuJq11EYENxVXpWgLgqj7Mk3HNKXOhCRovZnHs24-PB6Pn3wXSX6TR3yjJyjZWCbyUFWwhiuhjkOAGl84Yy0tBEKvexCGo4gZabnvY3QhVyqEMTMhYrQ60_JUwtTKpAZpw3-a0M8HbsRIzXa3mTLHZnWoL-cCCuTOdrOAcnLkLbEmcGgILMPp05HHFrsU7jQiG2RMoZoL_AGCpoXmuwAAAA&shmds=v1_ATWGeeNJb8XUgXPtJlkeDlbrA73EJm5eYfDabQBHJdJ5sHk0Bw&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=Software+Engineer&htidocid=C8-O3Oo_8QlZzDQ7AAAAAA%3D%3D,Software Engineer II - Reactjs Java,JPMorgan Chase Login
GE Vernova,Anywhere,Software Engineer,https://www.google.com/search?ibp=htl;jobs&q=Software+Engineer&htidocid=7VdAOS-XTTlXUQMAAAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xXMMQ7CMAwAQLH2CUxeQaUBJBaYqwIriLVyGpMGpXYVR1DewmeB5cYrPrNiUw-WnCMHF7nnFyaCmn1gogQrOIsFJUxdD8LQiPhI80Of86h7Y1Rj5TVjDl3VyWCEycpkHmL1T6v9rxsjZmq3u_VUjeyXi6aGGyWWJ0JgOL4dJbToSrhSRPbIWMKJXcAvsJ1QkZ4AAAA&shmds=v1_ATWGeeMu-oeyp3h7he1x7KcvL61yNFxkLmLpeEBgihl62kqgBw&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=Software+Engineer&htidocid=7VdAOS-XTTlXUQMAAAAAAA%3D%3D,Embedded Software Engineer,GE Vernova Careers
S&P Global,"Hyderabad, Telangana, India",Software Engineer,https://www.google.com/search?ibp=htl;jobs&q=Software+Engineer&htidocid=urrhaM1VKlmozkJ2AAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xWNsQ6CMBBA48onON1kokFqTFxkNqgTCe7kSs9SUu5IWyN-kP8pLm94w3vZd5WVjTzTGwPBha1jogB7qAKaFyaCOogNOC7qLhoiYeh6EIZKxHpal31KUzwrFaMvbEyYXFd0Miph0jKrQXT8o439cpj8kmyPp8NcTGx322ZTQ-VFowfHcP0YCqjR5PAgj2yRMYcbG4c_DVtzwagAAAA&shmds=v1_ATWGeePEiDwxDgkD708F6mXCfFT5_alQqbc0r6lS_9ZnTg8fRg&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=Software+Engineer&htidocid=urrhaM1VKlmozkJ2AAAAAA%3D%3D,Software Engineer - Graduate Program,S&P Global Careers
"48 Marvell India Pvt Ltd, Hyderabad","Hyderabad, Telangana, India",Software Engineer,https://www.google.com/search?ibp=htl;jobs&q=Software+Engineer&htidocid=yRLMTxqxU7OQyqheAAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_0WNwYrCQBBE2Ws-wVPfFiQmIi6InhSXdUVBcO_SSdpJZOwO002M_-eHOeJhL0XVg-Ilj49keZSz3TAQfLNrmCjALxsFhhHsUWP7VFiTC0SRbKUAJQxlDcLwI-I8DRa1WavzPFf1mVNDa8qslGsuTIX0-UUKfcVJ6-hpPRqdJl_jPmvZDVfTWfSEjryP4qpBOHQGO6tS2NwrClhgBQ3_jxT-yCM7ZEzfjyd4Cf9OxwAAAA&shmds=v1_ATWGeeNpIONwdD_RbnozBzH-Tc2ODJiMz3qRFBUmwYjr8tjxiQ&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=Software+Engineer&htidocid=yRLMTxqxU7OQyqheAAAAAA%3D%3D,Software Engineer Intern - Master's Degree,Workday
Chase- Candidate Experience page,"Hyderabad, Telangana, India",Software Engineer,https://www.google.com/search?ibp=htl;jobs&q=Software+Engineer&htidocid=wH8Nd0Otipx8zuv2AAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xWNvQrCMBRGcfURnO7kILYRwUUH0SL-4CAoOJbb5ppE6r0hCVpfzyezXc5wDh_f8DcYbs-EGq7ySB8MBDs2jokCZHDCNyq4fJMVhjFs7tdeSgWRMNQWOrsXMQ2NVjYlH5dKxdjkJiZMrs5reSlhqqRVT6lijzLa7sM3mKicL2Zt7tlM1oXFSBkUyNrpLsGu9RQccU3g0RA4hsNXU8AK9RRu1CAbZJzCsVvgHxpZ75LFAAAA&shmds=v1_ATWGeeMgnnuUZVpfMO9AcBilZHVKHtUkqcGaHtD2g3jK8I7BSQ&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=Software+Engineer&htidocid=wH8Nd0Otipx8zuv2AAAAAA%3D%3D,Lead Software Engineer - Java/ Python & AWS,JPMorgan Chase Login
Meta,

Save to Bronze Layer

- Create the schema if not exists

- Write the DataFrame as a Delta table to the Bronze layer

In [0]:
#Save to Bronze Layer
spark.sql("CREATE SCHEMA IF NOT EXISTS job_etl_bronze")

bronze_df.write \
    .format("delta") \
    .mode("append") \
    .option("mergeSchema", "true") \
    .saveAsTable("job_etl_bronze.jobs_raw")

print("Data written to Bronze Layer")


Data written to Bronze Layer


### 🥈 Silver Layer – Data Cleaning & Standardization (Job ETL Pipeline)

Read Raw Job Data from Bronze Layer

In [0]:
# Read Bronze Layer data
# Step 1: Read Bronze data
bronze_df = spark.table("job_etl_bronze.jobs_raw")

# Check total rows
print("Total rows in Bronze:", bronze_df.count())

# Preview data
bronze_df.display()



Total rows in Bronze: 475


Search_role apply_options company_name description detected_extensions extensions job_id location share_link thumbnail title via job_link job_snippet job_title ingestion_date Software Engineer List(Map(title -> ServiceNow Careers, link -> https://careers.servicenow.com/jobs/744000102045336/staff-software-engineer-python-golang/?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic), Map(title -> LinkedIn, link -> https://in.linkedin.com/jobs/view/staff-software-engineer-python-golang-at-servicenow-4346600441?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic), Map(title -> Talent.com, link -> https://in.talent.com/view?id=b7c92b84b5ac&utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic), Map(title -> Jooble, link -> https://in.jooble.org/jdp/2530133641092021705?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic), Map(title -> Recruit.net, link -> https://www.recruit.net/job/staff-software-engineer-python-golang-jobs/311DAED336A3030A?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic), Map(title -> Jobrapido.com, link -> https://in.jobrapido.com/jobpreview/8547077220034674688?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic)) ServiceNow Company Description

It all started in sunny San Diego, California in 2004 when a visionary engineer, Fred Luddy, saw the potential to transform how we work. Fast forward to today — ServiceNow stands as a global market leader, bringing innovative AI-enhanced technology to over 8,100 customers, including 85% of the Fortune 500®. Our intelligent cloud-based platform seamlessly connects people, systems, and processes to empower organizations to find smarter, faster, and better ways to work. But this is just the beginning of our journey. Join us as we pursue our purpose to make the world work better for everyone.
Job Description

Team overview

We are AI Services, a shared services platform team building solutions around Machine Learning and Generative AI capabilities. We build and evolve the AI platform, and partner with teams to build products and end-to-end AI-powered work experiences. As a Staff Machine Learning Software Engineer, you will play a major part in building solutions around Generative AI and Machine Learning (ML) capabilities that transform the user experience and workflow efficiency of our enterprise services. You will play a crucial role in developing and maintaining robust tools and services that enable customers to create, test, and deploy custom skills and prompts. You will also be key to ensuring seamless integration with other ServiceNow products and external LLM providers, such as Gemini, Azure and AWS.

By joining our team, you will play an important role in shaping the future of AI capabilities on the ServiceNow platform, ensuring that our customers can harness the power of generative AI to improve productivity and self-deflection.

What you get to do in this role:
• Build high-quality, clean, scalable and reusable code by enforcing best practices around software engineering architecture and processes (Code Reviews, Unit testing, etc.)
• Work with the product owners to understand detailed requirements and own your code from design, implementation, test automation and delivery of high-quality product to our users.
• Design software that is simple to use to allow customers to extend and customize the functionality to meet their specific needs
• Help design and implement new products and features while also enhancing the existing product suite
Qualifications

To be successful in this role you have:
• Experience in leveraging or critically thinking about how to integrate AI into work processes, decision-making, or problem-solving. This may include using AI-powered tools, automating workflows, analyzing AI-driven insights, or exploring AI's potential impact on the function or industry.
• 8+ years of related experien

Inspect Schema and Data Quality

In [0]:
# Check schema
bronze_df.printSchema()


root
 |-- Search_role: string (nullable = true)
 |-- apply_options: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)
 |-- company_name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- detected_extensions: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- extensions: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- job_id: string (nullable = true)
 |-- location: string (nullable = true)
 |-- share_link: string (nullable = true)
 |-- thumbnail: string (nullable = true)
 |-- title: string (nullable = true)
 |-- via: string (nullable = true)
 |-- job_link: string (nullable = true)
 |-- job_snippet: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- ingestion_date: date (nullable = true)



Select Business-Relevant Columns

In [0]:
from pyspark.sql.functions import col

silver_df = bronze_df.select(
    col("title").alias("job_title"),
    col("company_name"),
    col("location"),
    col("via").alias("job_source"),
    col("share_link").alias("job_apply_link"),
    col("search_role")
)

silver_df.display()


job_title,company_name,location,job_source,job_apply_link,search_role
Staff Software Engineer - Python & Golang,ServiceNow,"Hyderabad, Telangana, India",ServiceNow Careers,https://www.google.com/search?ibp=htl;jobs&q=Software+Engineer&htidocid=PCIMKdItwxs7RmWhAAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xWNsQ6CMBBA48onON1kokEwJi66uBjUwZjgTq7laGvwjrSNwC_5lcLyhveGl_wWybmM2DRQShN79AQXNo6JPGzhOUYrDCsopEU2k7mLgkDotYUpFCKmpeXJxtiFY56H0GYmRIxOZ1o-uTApGfK3qDCjCnYadC1GqvaH3ZB1bDbrkvzXaXpID47hOtbkUWGdwovmKTKmcOPa4R-diUZBrQAAAA&shmds=v1_ATWGeePv2NJedJdFralF3FTkQ0wpIfl1KYUiOmnSHwdY1DIEZA&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=Software+Engineer&htidocid=PCIMKdItwxs7RmWhAAAAAA%3D%3D,Software Engineer
Software Engineer II - Reactjs Java,JPMC Candidate Experience page,"Hyderabad, Telangana, India",JPMorgan Chase Login,https://www.google.com/search?ibp=htl;jobs&q=Software+Engineer&htidocid=C8-O3Oo_8QlZzDQ7AAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xWNQQrCMBBFcdsjuJq11EYENxVXpWgLgqj7Mk3HNKXOhCRovZnHs24-PB6Pn3wXSX6TR3yjJyjZWCbyUFWwhiuhjkOAGl84Yy0tBEKvexCGo4gZabnvY3QhVyqEMTMhYrQ60_JUwtTKpAZpw3-a0M8HbsRIzXa3mTLHZnWoL-cCCuTOdrOAcnLkLbEmcGgILMPp05HHFrsU7jQiG2RMoZoL_AGCpoXmuwAAAA&shmds=v1_ATWGeeNJb8XUgXPtJlkeDlbrA73EJm5eYfDabQBHJdJ5sHk0Bw&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=Software+Engineer&htidocid=C8-O3Oo_8QlZzDQ7AAAAAA%3D%3D,Software Engineer
Embedded Software Engineer,GE Vernova,Anywhere,GE Vernova Careers,https://www.google.com/search?ibp=htl;jobs&q=Software+Engineer&htidocid=7VdAOS-XTTlXUQMAAAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xXMMQ7CMAwAQLH2CUxeQaUBJBaYqwIriLVyGpMGpXYVR1DewmeB5cYrPrNiUw-WnCMHF7nnFyaCmn1gogQrOIsFJUxdD8LQiPhI80Of86h7Y1Rj5TVjDl3VyWCEycpkHmL1T6v9rxsjZmq3u_VUjeyXi6aGGyWWJ0JgOL4dJbToSrhSRPbIWMKJXcAvsJ1QkZ4AAAA&shmds=v1_ATWGeeMu-oeyp3h7he1x7KcvL61yNFxkLmLpeEBgihl62kqgBw&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=Software+Engineer&htidocid=7VdAOS-XTTlXUQMAAAAAAA%3D%3D,Software Engineer
Software Engineer - Graduate Program,S&P Global,"Hyderabad, Telangana, India",S&P Global Careers,https://www.google.com/search?ibp=htl;jobs&q=Software+Engineer&htidocid=urrhaM1VKlmozkJ2AAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xWNsQ6CMBBA48onON1kokFqTFxkNqgTCe7kSs9SUu5IWyN-kP8pLm94w3vZd5WVjTzTGwPBha1jogB7qAKaFyaCOogNOC7qLhoiYeh6EIZKxHpal31KUzwrFaMvbEyYXFd0Miph0jKrQXT8o439cpj8kmyPp8NcTGx322ZTQ-VFowfHcP0YCqjR5PAgj2yRMYcbG4c_DVtzwagAAAA&shmds=v1_ATWGeePEiDwxDgkD708F6mXCfFT5_alQqbc0r6lS_9ZnTg8fRg&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=Software+Engineer&htidocid=urrhaM1VKlmozkJ2AAAAAA%3D%3D,Software Engineer
Software Engineer Intern - Master's Degree,"48 Marvell India Pvt Ltd, Hyderabad","Hyderabad, Telangana, India",Workday,https://www.google.com/search?ibp=htl;jobs&q=Software+Engineer&htidocid=yRLMTxqxU7OQyqheAAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_0WNwYrCQBBE2Ws-wVPfFiQmIi6InhSXdUVBcO_SSdpJZOwO002M_-eHOeJhL0XVg-Ilj49keZSz3TAQfLNrmCjALxsFhhHsUWP7VFiTC0SRbKUAJQxlDcLwI-I8DRa1WavzPFf1mVNDa8qslGsuTIX0-UUKfcVJ6-hpPRqdJl_jPmvZDVfTWfSEjryP4qpBOHQGO6tS2NwrClhgBQ3_jxT-yCM7ZEzfjyd4Cf9OxwAAAA&shmds=v1_ATWGeeNpIONwdD_RbnozBzH-Tc2ODJiMz3qRFBUmwYjr8tjxiQ&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=Software+Engineer&htidocid=yRLMTxqxU7OQyqheAAAAAA%3D%3D,Software Engineer
Lead Software Engineer - Java/ Python & AWS,Chase- Candidate Experience page,"Hyderabad, Telangana, India",JPMorgan Chase Login,https://www.google.com/search?ibp=htl;jobs&q=Software+Engineer&htidocid=wH8Nd0Otipx8zuv2AAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xWNvQrCMBRGcfURnO7kILYRwUUH0SL-4CAoOJbb5ppE6r0hCVpfzyezXc5wDh_f8DcYbs-EGq7ySB8MBDs2jokCZHDCNyq4fJMVhjFs7tdeSgWRMNQWOrsXMQ2NVjYlH5dKxdjkJiZMrs5reSlhqqRVT6lijzLa7sM3mKicL2Zt7tlM1oXFSBkUyNrpLsGu9RQccU3g0RA4hsNXU8AK9RRu1CAbZJzCsVvgHxpZ75LFAAAA&shmds=v1_ATWGeeMgnnuUZVpfMO9AcBilZHVKHtUkqcGaHtD2g3jK8I7BSQ&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=Software+Engineer&htidocid=wH8Nd0Otipx8zuv2AAAAAA%3D%3D,Software

Handle Null & Empty Values - Remove Null Records

In [0]:
silver_df = silver_df.dropna(
    subset=["job_title", "job_apply_link", "company_name"]
)

print("After removing nulls:", silver_df.count())


After removing nulls: 290


Deduplicate Job Listings - Remove Duplicate Job Listings


In [0]:
silver_df = silver_df.dropDuplicates(
    ["job_title", "company_name", "job_apply_link"]
)

print("After removing duplicates:", silver_df.count())


After removing duplicates: 110


Basic Data Standardization - Standardize Text Columns

In [0]:
from pyspark.sql.functions import trim, lower

silver_df = silver_df.withColumn("job_title", trim(col("job_title"))) \
                     .withColumn("company_name", trim(col("company_name"))) \
                     .withColumn("search_role", lower(col("search_role")))
silver_df.display()

job_title,company_name,location,job_source,job_apply_link,search_role
Infrastructure Architect IT,Infosys,India,Shine,https://www.google.com/search?ibp=htl;jobs&q=IT+Jobs&htidocid=NjlyRiquNJUh8raqAAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xXNIQ_CMBBA4WCn0KjTJLRkCQYUihSNX9pytCXbXdO7JUPzxxnmic-87rvpekev5kXbHHVuCNcWc1GMCu4BB7hzAEG_IjDBjTmNuLtk1Spna0VGk0S9lmgiT5YJAy_2zUH-GST7hnX0ikN_Oi6mUtpv1yHLR6AQOHoW_wMGcEothgAAAA&shmds=v1_ATWGeeMZ5q-LTANCgPzTbGSZUAl4KpwDHHPxx6ePrYZlmIL3OQ&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=IT+Jobs&htidocid=NjlyRiquNJUh8raqAAAAAA%3D%3D,it jobs
Senior Associate Accelerated Solutions (M&A IT),PricewaterhouseCoopers Acceleration Centers,India,Shine,https://www.google.com/search?ibp=htl;jobs&q=IT+Jobs&htidocid=3Na0ST8PT2L3aqMiAAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_0WOsQrCMBRFce0nOL1JVLAR0UWn0qFUEIS6lzQ-mkh8r-Sl2I_z40wnl8vlDIeTfRdZ1SA5DlCIsHE6IkBhDHoM6T-hYT9GxySwvq0KqB8b2MGVOxDUwVhggoq597i82BgHOSsl4vNeoo7O5Ibfigk7ntSLO5mnFasDDj7p28NpP-UD9dvjPTiDn8SC5VGwZB4wyD8lJUCJFGfoCGp6Ov0DBIzADr8AAAA&shmds=v1_ATWGeeP9GxMQa-9LGvnZLkFiTK9CEIMgEl1KRR29xTyA0lslvA&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=IT+Jobs&htidocid=3Na0ST8PT2L3aqMiAAAAAA%3D%3D,it jobs
Senior Software Engineer II - JAVA Backend,UnitedHealth Group,"Hyderabad, Telangana, India",Work Here - UnitedHealth Group,https://www.google.com/search?ibp=htl;jobs&q=Software+Engineer&htidocid=2RW1iJVjbrC9QXZsAAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xXNsQ6CMBCA4bjyCE43G6Rq4qITJgZwRV3JQc9SrXekrRGfyxdUlj_5pj_5zpK8JrbioZZbfKMnOLKxTOShqmAJp_yawwG7B7GeKC0EQt_1IAyFiHE03_cxDmGnVAguMyFitF3WyVMJUyujuksbpjSh_w8Gh5GazXY1ZgObxfrCNpIuCV3sofDyGsAylB9NHlvUKZzJIRtkTKFibfEHZGL0KrYAAAA&shmds=v1_ATWGeePGiRMCyokdPT60M7m8M3J-bcb2SmVPtU6h8Cm0INPixw&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=Software+Engineer&htidocid=2RW1iJVjbrC9QXZsAAAAAA%3D%3D,software engineer
Head - Strategic Marketing - IT,ResourceTree Global Services Pvt Ltd.,India,Jobaaj,https://www.google.com/search?ibp=htl;jobs&q=IT+Jobs&htidocid=vOpJyaYa3U445HVDAAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xXMsQrCMBCAYVz7CE43C6Yi6KAPUCsKot3LJT3SaMyV3Fn6Qr6ndfmGf_iL76LYnwg7WMNDMyr54OCK-UUakp9r3cyc2YIQZtcDJ6iYfaTlsVcd5FCWItF4UdTgjON3yYksT-WTrfxppcdMQ5zn7Xa3mcyQ_MrcSfiTHTWZCKrIFiM8KI_BkcBtVLhoZyAkqFMX8Ad31Q8DqAAAAA&shmds=v1_ATWGeeNLt6NLsZ5_-6ZiuZgKhKLsTuSS1dVG71537K-kkllp-w&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=IT+Jobs&htidocid=vOpJyaYa3U445HVDAAAAAA%3D%3D,it jobs
Network Administrator(IT),Quadron iSolutions,India,Glassdoor,https://www.google.com/search?ibp=htl;jobs&q=IT+Jobs&htidocid=GvF2hymk74aQp_PXAAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xXMsQrCQAwAUFz7CbpkVMFeEVzs5CR1EET3krZHG70mxyXFfoQfrS5vfNlnkRVXb29JLzh1IzGpJTRJ6-qxgR1cpAH1mNoBhOEs0ge_LAezqEfnVEPeq6FRm7cyOmHfyOye0uifWgdMPgY0X-8PxZxH7rer24Rd-mV0lzAZCSsQQ8Ud4RdeKo7bjwAAAA&shmds=v1_ATWGeePhT1vnK85iz2KKAUg7nXG-vNEtHCKYBZHF2Fh54qKvdw&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=IT+Jobs&htidocid=GvF2hymk74aQp_PXAAAAAA%3D%3D,it jobs
Financial Master Data Analyst,Warner Bros. Discovery,"Hyderabad, Telangana, India",WBD Careers - Warner Bros. Discovery,https://www.google.com/search?ibp=htl;jobs&q=Data+Analyst&htidocid=MFmZLfNtEdsNr_nSAAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xXNPQrCQBAGUGxzBKupRRMRbbRSgn9gJ1jKl82wWVlnws4i5kqeUm1e-4rPqFjug0BcQKQLLHOiGhm0FcTBMs3orA0ZI7mOVOig6iOPN13Ova2ryiyW3jJycKXTZ6XCjb6rhzb2524dEvcRme-L1fxd9uInqxuS_KJdUiupDub0xWmgIHQcWk5o0E7pyhHiIZjSSdqAL3xeRA6tAAAA&shmds=v1_ATWGeePjnQgx9Du20_sxPP17bR_bDIAok5cW7PINbPZG_X6B4A&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=Data+Analyst&htidocid=MFmZLfNtEdsNr_nSAAAAAA%3D%3D,data analyst
Digital Transformation Head-Engineering-BFSI- Bangalore,Spot Your Leaders & Consulting,الهند,Shine,https://www.google.com/search?ibp=htl;jobs&q=Technology+Jobs&htidocid=RZLUhdhJ4RJjlzdsAAAAAA%3D%3D&hl=ar-QA&shndl=37&shmd=H4sIAAAAAAAA_x3OvUpDQRCGYWxzCVZTWQjnHBVstIvxD-xiYxX2JONmZTNz2NlASgMh_uB1GCGIrd6HxUzrlXiS5oO

Write Cleaned Data to Silver Layer

In [0]:
# Create Silver schema
spark.sql("CREATE SCHEMA IF NOT EXISTS job_etl_silver")

# Write Silver table
silver_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("job_etl_silver.jobs_clean")

print("Data written to Silver Layer")


Data written to Silver Layer


###  🥇 Gold Layer: Business Aggregations & Analytics

Read Data from Silver Layer

In [0]:
# Read Silver layer data
silver_df = spark.read.table("job_etl_silver.jobs_clean")

print("Total records in Silver:", silver_df.count())
silver_df.display()


Total records in Silver: 110


job_title,company_name,location,job_source,job_apply_link,search_role
Infrastructure Architect IT,Infosys,India,Shine,https://www.google.com/search?ibp=htl;jobs&q=IT+Jobs&htidocid=NjlyRiquNJUh8raqAAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xXNIQ_CMBBA4WCn0KjTJLRkCQYUihSNX9pytCXbXdO7JUPzxxnmic-87rvpekev5kXbHHVuCNcWc1GMCu4BB7hzAEG_IjDBjTmNuLtk1Spna0VGk0S9lmgiT5YJAy_2zUH-GST7hnX0ikN_Oi6mUtpv1yHLR6AQOHoW_wMGcEothgAAAA&shmds=v1_ATWGeeMZ5q-LTANCgPzTbGSZUAl4KpwDHHPxx6ePrYZlmIL3OQ&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=IT+Jobs&htidocid=NjlyRiquNJUh8raqAAAAAA%3D%3D,it jobs
Senior Associate Accelerated Solutions (M&A IT),PricewaterhouseCoopers Acceleration Centers,India,Shine,https://www.google.com/search?ibp=htl;jobs&q=IT+Jobs&htidocid=3Na0ST8PT2L3aqMiAAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_0WOsQrCMBRFce0nOL1JVLAR0UWn0qFUEIS6lzQ-mkh8r-Sl2I_z40wnl8vlDIeTfRdZ1SA5DlCIsHE6IkBhDHoM6T-hYT9GxySwvq0KqB8b2MGVOxDUwVhggoq597i82BgHOSsl4vNeoo7O5Ibfigk7ntSLO5mnFasDDj7p28NpP-UD9dvjPTiDn8SC5VGwZB4wyD8lJUCJFGfoCGp6Ov0DBIzADr8AAAA&shmds=v1_ATWGeeP9GxMQa-9LGvnZLkFiTK9CEIMgEl1KRR29xTyA0lslvA&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=IT+Jobs&htidocid=3Na0ST8PT2L3aqMiAAAAAA%3D%3D,it jobs
Senior Software Engineer II - JAVA Backend,UnitedHealth Group,"Hyderabad, Telangana, India",Work Here - UnitedHealth Group,https://www.google.com/search?ibp=htl;jobs&q=Software+Engineer&htidocid=2RW1iJVjbrC9QXZsAAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xXNsQ6CMBCA4bjyCE43G6Rq4qITJgZwRV3JQc9SrXekrRGfyxdUlj_5pj_5zpK8JrbioZZbfKMnOLKxTOShqmAJp_yawwG7B7GeKC0EQt_1IAyFiHE03_cxDmGnVAguMyFitF3WyVMJUyujuksbpjSh_w8Gh5GazXY1ZgObxfrCNpIuCV3sofDyGsAylB9NHlvUKZzJIRtkTKFibfEHZGL0KrYAAAA&shmds=v1_ATWGeePGiRMCyokdPT60M7m8M3J-bcb2SmVPtU6h8Cm0INPixw&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=Software+Engineer&htidocid=2RW1iJVjbrC9QXZsAAAAAA%3D%3D,software engineer
Head - Strategic Marketing - IT,ResourceTree Global Services Pvt Ltd.,India,Jobaaj,https://www.google.com/search?ibp=htl;jobs&q=IT+Jobs&htidocid=vOpJyaYa3U445HVDAAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xXMsQrCMBCAYVz7CE43C6Yi6KAPUCsKot3LJT3SaMyV3Fn6Qr6ndfmGf_iL76LYnwg7WMNDMyr54OCK-UUakp9r3cyc2YIQZtcDJ6iYfaTlsVcd5FCWItF4UdTgjON3yYksT-WTrfxppcdMQ5zn7Xa3mcyQ_MrcSfiTHTWZCKrIFiM8KI_BkcBtVLhoZyAkqFMX8Ad31Q8DqAAAAA&shmds=v1_ATWGeeNLt6NLsZ5_-6ZiuZgKhKLsTuSS1dVG71537K-kkllp-w&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=IT+Jobs&htidocid=vOpJyaYa3U445HVDAAAAAA%3D%3D,it jobs
Network Administrator(IT),Quadron iSolutions,India,Glassdoor,https://www.google.com/search?ibp=htl;jobs&q=IT+Jobs&htidocid=GvF2hymk74aQp_PXAAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xXMsQrCQAwAUFz7CbpkVMFeEVzs5CR1EET3krZHG70mxyXFfoQfrS5vfNlnkRVXb29JLzh1IzGpJTRJ6-qxgR1cpAH1mNoBhOEs0ge_LAezqEfnVEPeq6FRm7cyOmHfyOye0uifWgdMPgY0X-8PxZxH7rer24Rd-mV0lzAZCSsQQ8Ud4RdeKo7bjwAAAA&shmds=v1_ATWGeePhT1vnK85iz2KKAUg7nXG-vNEtHCKYBZHF2Fh54qKvdw&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=IT+Jobs&htidocid=GvF2hymk74aQp_PXAAAAAA%3D%3D,it jobs
Financial Master Data Analyst,Warner Bros. Discovery,"Hyderabad, Telangana, India",WBD Careers - Warner Bros. Discovery,https://www.google.com/search?ibp=htl;jobs&q=Data+Analyst&htidocid=MFmZLfNtEdsNr_nSAAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xXNPQrCQBAGUGxzBKupRRMRbbRSgn9gJ1jKl82wWVlnws4i5kqeUm1e-4rPqFjug0BcQKQLLHOiGhm0FcTBMs3orA0ZI7mOVOig6iOPN13Ova2ryiyW3jJycKXTZ6XCjb6rhzb2524dEvcRme-L1fxd9uInqxuS_KJdUiupDub0xWmgIHQcWk5o0E7pyhHiIZjSSdqAL3xeRA6tAAAA&shmds=v1_ATWGeePjnQgx9Du20_sxPP17bR_bDIAok5cW7PINbPZG_X6B4A&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=Data+Analyst&htidocid=MFmZLfNtEdsNr_nSAAAAAA%3D%3D,data analyst
Digital Transformation Head-Engineering-BFSI- Bangalore,Spot Your Leaders & Consulting,الهند,Shine,https://www.google.com/search?ibp=htl;jobs&q=Technology+Jobs&htidocid=RZLUhdhJ4RJjlzdsAAAAAA%3D%3D&hl=ar-QA&shndl=37&shmd=H4sIAAAAAAAA_x3OvUpDQRCGYWxzCVZTWQjnHBVstIvxD-xiYxX2JONmZTNz2NlASgMh_uB1GCGIrd6HxUzrlXiS5oO

## Jobs per Role in Top 5 Locations

Which roles are most in demand in the top 5 cities?

In [0]:
from pyspark.sql.functions import col

# Top 5 locations
top_locations = spark.table("job_etl_gold.jobs_by_location") \
                     .orderBy("total_jobs", ascending=False) \
                     .limit(5) \
                     .select("location")

# Filter jobs in top locations and count per role
spark.table("job_etl_silver.jobs_clean") \
     .join(top_locations, "location") \
     .groupBy("search_role", "location") \
     .count() \
     .orderBy("count", ascending=False) \
     .display()


search_role,location,count
data analyst,"Hyderabad, Telangana, India",9
it jobs,India,9
devops engineer,India,7
software engineer,"Hyderabad, Telangana, India",7
technology jobs,الهند,7
full stack developer,India,7
data scientist,"Hyderabad, Telangana, India",4
frontend developer,India,4
backend developer,India,4
data analyst,Anywhere,3


## Top Companies Hiring for a Specific Role

Which companies are hiring the most for ‘Data Engineer’?

In [0]:
role = "data engineer"

spark.table("job_etl_silver.jobs_clean") \
     .filter(col("search_role") == role) \
     .groupBy("company_name") \
     .count() \
     .orderBy("count", ascending=False) \
     .display()
 



company_name,count
Mastercard,1
JP Morgan Chase,1
IntraEdge,1
FIS Management Services LLC,1
UPS,1
HMH,1
Cognizant,1
Warner Bros. Discovery,1
Accenture,1
Prophecy Technologies,1


## Roles Available in a Specific Location

What roles are available in Hyderabad?

In [0]:
# Import required function
from pyspark.sql.functions import lower, col

# Set your location
location = "hyderabad, telangana, india"

# Query Gold insights: Jobs per role in this location
spark.table("job_etl_silver.jobs_clean") \
     .filter(lower(col("location")) == location.lower()) \
     .groupBy("search_role") \
     .count() \
     .orderBy("count", ascending=False) \
     .display()



search_role,count
data analyst,9
software engineer,7
data scientist,4
backend developer,2
data engineer,1


## Companies with Most Diverse Roles

Which companies are hiring for the most different types of roles?

In [0]:
from pyspark.sql.functions import countDistinct, col

# Top companies by number of unique roles
spark.table("job_etl_silver.jobs_clean") \
     .groupBy("company_name") \
     .agg(countDistinct("search_role").alias("unique_roles")) \
     .orderBy("unique_roles", ascending=False) \
     .display()



company_name,unique_roles
Accenture,4
Oracle,3
Jobgether,3
AJIVA INFOTECH PVT LTD,2
Smart Working Solutions,2
Zoom,2
Egnyte,2
Infosys,2
Cognizant,2
Warner Bros. Discovery,2


## Roles That Have Both High Hiring and High Locations

Which roles appear in the most locations and have high job counts?

In [0]:
from pyspark.sql.functions import count, countDistinct, col

spark.table("job_etl_silver.jobs_clean") \
     .groupBy("search_role") \
     .agg(
         count("job_title").alias("total_jobs"),
         countDistinct("location").alias("locations_count")
     ) \
     .orderBy(col("total_jobs").desc(), col("locations_count").desc()) \
     .display()



search_role,total_jobs,locations_count
data analyst,20,6
data engineer,10,8
backend developer,10,5
frontend developer,10,4
data scientist,10,4
devops engineer,10,3
software engineer,10,3
technology jobs,10,3
full stack developer,10,3
it jobs,10,2


## Saving Aggregated Job Insights to the Gold Layer

In [0]:
#Gold Layer: Persisting Aggregated Job Analytics Tables
# Create Gold schema
spark.sql("CREATE SCHEMA IF NOT EXISTS job_etl_gold")

# Save jobs by role
jobs_by_role.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("job_etl_gold.jobs_by_role")

# Save jobs by company
jobs_by_company.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("job_etl_gold.jobs_by_company")

# Save jobs by location
jobs_by_location.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("job_etl_gold.jobs_by_location")

print("All Gold Layer tables saved successfully")



All Gold Layer tables saved successfully


## 🥇 Creating Final Gold Jobs Table

In [0]:
from pyspark.sql.functions import col

final_gold_df = spark.table("job_etl_silver.jobs_clean") \
    .select(
        col("job_title"),
        col("company_name"),
        col("location"),
        col("search_role"),
        col("job_apply_link"),
        col("job_source")
    )

final_gold_df.display()

#Save This as Gold Table
final_gold_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("job_etl_gold.jobs_final")

print("Final Gold Jobs table saved successfully")



job_title,company_name,location,search_role,job_apply_link,job_source
Infrastructure Architect IT,Infosys,India,it jobs,https://www.google.com/search?ibp=htl;jobs&q=IT+Jobs&htidocid=NjlyRiquNJUh8raqAAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xXNIQ_CMBBA4WCn0KjTJLRkCQYUihSNX9pytCXbXdO7JUPzxxnmic-87rvpekev5kXbHHVuCNcWc1GMCu4BB7hzAEG_IjDBjTmNuLtk1Spna0VGk0S9lmgiT5YJAy_2zUH-GST7hnX0ikN_Oi6mUtpv1yHLR6AQOHoW_wMGcEothgAAAA&shmds=v1_ATWGeeMZ5q-LTANCgPzTbGSZUAl4KpwDHHPxx6ePrYZlmIL3OQ&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=IT+Jobs&htidocid=NjlyRiquNJUh8raqAAAAAA%3D%3D,Shine
Senior Associate Accelerated Solutions (M&A IT),PricewaterhouseCoopers Acceleration Centers,India,it jobs,https://www.google.com/search?ibp=htl;jobs&q=IT+Jobs&htidocid=3Na0ST8PT2L3aqMiAAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_0WOsQrCMBRFce0nOL1JVLAR0UWn0qFUEIS6lzQ-mkh8r-Sl2I_z40wnl8vlDIeTfRdZ1SA5DlCIsHE6IkBhDHoM6T-hYT9GxySwvq0KqB8b2MGVOxDUwVhggoq597i82BgHOSsl4vNeoo7O5Ibfigk7ntSLO5mnFasDDj7p28NpP-UD9dvjPTiDn8SC5VGwZB4wyD8lJUCJFGfoCGp6Ov0DBIzADr8AAAA&shmds=v1_ATWGeeP9GxMQa-9LGvnZLkFiTK9CEIMgEl1KRR29xTyA0lslvA&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=IT+Jobs&htidocid=3Na0ST8PT2L3aqMiAAAAAA%3D%3D,Shine
Senior Software Engineer II - JAVA Backend,UnitedHealth Group,"Hyderabad, Telangana, India",software engineer,https://www.google.com/search?ibp=htl;jobs&q=Software+Engineer&htidocid=2RW1iJVjbrC9QXZsAAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xXNsQ6CMBCA4bjyCE43G6Rq4qITJgZwRV3JQc9SrXekrRGfyxdUlj_5pj_5zpK8JrbioZZbfKMnOLKxTOShqmAJp_yawwG7B7GeKC0EQt_1IAyFiHE03_cxDmGnVAguMyFitF3WyVMJUyujuksbpjSh_w8Gh5GazXY1ZgObxfrCNpIuCV3sofDyGsAylB9NHlvUKZzJIRtkTKFibfEHZGL0KrYAAAA&shmds=v1_ATWGeePGiRMCyokdPT60M7m8M3J-bcb2SmVPtU6h8Cm0INPixw&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=Software+Engineer&htidocid=2RW1iJVjbrC9QXZsAAAAAA%3D%3D,Work Here - UnitedHealth Group
Head - Strategic Marketing - IT,ResourceTree Global Services Pvt Ltd.,India,it jobs,https://www.google.com/search?ibp=htl;jobs&q=IT+Jobs&htidocid=vOpJyaYa3U445HVDAAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xXMsQrCMBCAYVz7CE43C6Yi6KAPUCsKot3LJT3SaMyV3Fn6Qr6ndfmGf_iL76LYnwg7WMNDMyr54OCK-UUakp9r3cyc2YIQZtcDJ6iYfaTlsVcd5FCWItF4UdTgjON3yYksT-WTrfxppcdMQ5zn7Xa3mcyQ_MrcSfiTHTWZCKrIFiM8KI_BkcBtVLhoZyAkqFMX8Ad31Q8DqAAAAA&shmds=v1_ATWGeeNLt6NLsZ5_-6ZiuZgKhKLsTuSS1dVG71537K-kkllp-w&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=IT+Jobs&htidocid=vOpJyaYa3U445HVDAAAAAA%3D%3D,Jobaaj
Network Administrator(IT),Quadron iSolutions,India,it jobs,https://www.google.com/search?ibp=htl;jobs&q=IT+Jobs&htidocid=GvF2hymk74aQp_PXAAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xXMsQrCQAwAUFz7CbpkVMFeEVzs5CR1EET3krZHG70mxyXFfoQfrS5vfNlnkRVXb29JLzh1IzGpJTRJ6-qxgR1cpAH1mNoBhOEs0ge_LAezqEfnVEPeq6FRm7cyOmHfyOye0uifWgdMPgY0X-8PxZxH7rer24Rd-mV0lzAZCSsQQ8Ud4RdeKo7bjwAAAA&shmds=v1_ATWGeePhT1vnK85iz2KKAUg7nXG-vNEtHCKYBZHF2Fh54qKvdw&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=IT+Jobs&htidocid=GvF2hymk74aQp_PXAAAAAA%3D%3D,Glassdoor
Financial Master Data Analyst,Warner Bros. Discovery,"Hyderabad, Telangana, India",data analyst,https://www.google.com/search?ibp=htl;jobs&q=Data+Analyst&htidocid=MFmZLfNtEdsNr_nSAAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xXNPQrCQBAGUGxzBKupRRMRbbRSgn9gJ1jKl82wWVlnws4i5kqeUm1e-4rPqFjug0BcQKQLLHOiGhm0FcTBMs3orA0ZI7mOVOig6iOPN13Ova2ryiyW3jJycKXTZ6XCjb6rhzb2524dEvcRme-L1fxd9uInqxuS_KJdUiupDub0xWmgIHQcWk5o0E7pyhHiIZjSSdqAL3xeRA6tAAAA&shmds=v1_ATWGeePjnQgx9Du20_sxPP17bR_bDIAok5cW7PINbPZG_X6B4A&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=Data+Analyst&htidocid=MFmZLfNtEdsNr_nSAAAAAA%3D%3D,WBD Careers - Warner Bros. Discovery
Digital Transformation Head-Engineering-BFSI- Bangalore,Spot Your Leaders & Consulting,الهند,technology jobs,https://www.google.com/search?ibp=htl;jobs&q=Technology+Jobs&htidocid=RZLUhdhJ4RJjlzdsAAAAAA%3D%3D&hl=ar-QA&shndl=37&shmd=H4sIAAAAAAAA_x3OvUpDQRCGYWxzCVZTWQjnHBVstIvxD-xiYxX2JONmZTNz2NlASgMh_uB1GCGIrd6Hx

Final Gold Jobs table saved successfully


## View Gold table using SQL

In [0]:
%sql
SELECT *
FROM job_etl_gold.jobs_final;


job_title,company_name,location,search_role,job_apply_link,job_source
Infrastructure Architect IT,Infosys,India,it jobs,https://www.google.com/search?ibp=htl;jobs&q=IT+Jobs&htidocid=NjlyRiquNJUh8raqAAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xXNIQ_CMBBA4WCn0KjTJLRkCQYUihSNX9pytCXbXdO7JUPzxxnmic-87rvpekev5kXbHHVuCNcWc1GMCu4BB7hzAEG_IjDBjTmNuLtk1Spna0VGk0S9lmgiT5YJAy_2zUH-GST7hnX0ikN_Oi6mUtpv1yHLR6AQOHoW_wMGcEothgAAAA&shmds=v1_ATWGeeMZ5q-LTANCgPzTbGSZUAl4KpwDHHPxx6ePrYZlmIL3OQ&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=IT+Jobs&htidocid=NjlyRiquNJUh8raqAAAAAA%3D%3D,Shine
Senior Associate Accelerated Solutions (M&A IT),PricewaterhouseCoopers Acceleration Centers,India,it jobs,https://www.google.com/search?ibp=htl;jobs&q=IT+Jobs&htidocid=3Na0ST8PT2L3aqMiAAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_0WOsQrCMBRFce0nOL1JVLAR0UWn0qFUEIS6lzQ-mkh8r-Sl2I_z40wnl8vlDIeTfRdZ1SA5DlCIsHE6IkBhDHoM6T-hYT9GxySwvq0KqB8b2MGVOxDUwVhggoq597i82BgHOSsl4vNeoo7O5Ibfigk7ntSLO5mnFasDDj7p28NpP-UD9dvjPTiDn8SC5VGwZB4wyD8lJUCJFGfoCGp6Ov0DBIzADr8AAAA&shmds=v1_ATWGeeP9GxMQa-9LGvnZLkFiTK9CEIMgEl1KRR29xTyA0lslvA&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=IT+Jobs&htidocid=3Na0ST8PT2L3aqMiAAAAAA%3D%3D,Shine
Senior Software Engineer II - JAVA Backend,UnitedHealth Group,"Hyderabad, Telangana, India",software engineer,https://www.google.com/search?ibp=htl;jobs&q=Software+Engineer&htidocid=2RW1iJVjbrC9QXZsAAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xXNsQ6CMBCA4bjyCE43G6Rq4qITJgZwRV3JQc9SrXekrRGfyxdUlj_5pj_5zpK8JrbioZZbfKMnOLKxTOShqmAJp_yawwG7B7GeKC0EQt_1IAyFiHE03_cxDmGnVAguMyFitF3WyVMJUyujuksbpjSh_w8Gh5GazXY1ZgObxfrCNpIuCV3sofDyGsAylB9NHlvUKZzJIRtkTKFibfEHZGL0KrYAAAA&shmds=v1_ATWGeePGiRMCyokdPT60M7m8M3J-bcb2SmVPtU6h8Cm0INPixw&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=Software+Engineer&htidocid=2RW1iJVjbrC9QXZsAAAAAA%3D%3D,Work Here - UnitedHealth Group
Head - Strategic Marketing - IT,ResourceTree Global Services Pvt Ltd.,India,it jobs,https://www.google.com/search?ibp=htl;jobs&q=IT+Jobs&htidocid=vOpJyaYa3U445HVDAAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xXMsQrCMBCAYVz7CE43C6Yi6KAPUCsKot3LJT3SaMyV3Fn6Qr6ndfmGf_iL76LYnwg7WMNDMyr54OCK-UUakp9r3cyc2YIQZtcDJ6iYfaTlsVcd5FCWItF4UdTgjON3yYksT-WTrfxppcdMQ5zn7Xa3mcyQ_MrcSfiTHTWZCKrIFiM8KI_BkcBtVLhoZyAkqFMX8Ad31Q8DqAAAAA&shmds=v1_ATWGeeNLt6NLsZ5_-6ZiuZgKhKLsTuSS1dVG71537K-kkllp-w&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=IT+Jobs&htidocid=vOpJyaYa3U445HVDAAAAAA%3D%3D,Jobaaj
Network Administrator(IT),Quadron iSolutions,India,it jobs,https://www.google.com/search?ibp=htl;jobs&q=IT+Jobs&htidocid=GvF2hymk74aQp_PXAAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xXMsQrCQAwAUFz7CbpkVMFeEVzs5CR1EET3krZHG70mxyXFfoQfrS5vfNlnkRVXb29JLzh1IzGpJTRJ6-qxgR1cpAH1mNoBhOEs0ge_LAezqEfnVEPeq6FRm7cyOmHfyOye0uifWgdMPgY0X-8PxZxH7rer24Rd-mV0lzAZCSsQQ8Ud4RdeKo7bjwAAAA&shmds=v1_ATWGeePhT1vnK85iz2KKAUg7nXG-vNEtHCKYBZHF2Fh54qKvdw&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=IT+Jobs&htidocid=GvF2hymk74aQp_PXAAAAAA%3D%3D,Glassdoor
Financial Master Data Analyst,Warner Bros. Discovery,"Hyderabad, Telangana, India",data analyst,https://www.google.com/search?ibp=htl;jobs&q=Data+Analyst&htidocid=MFmZLfNtEdsNr_nSAAAAAA%3D%3D&hl=en-US&shndl=37&shmd=H4sIAAAAAAAA_xXNPQrCQBAGUGxzBKupRRMRbbRSgn9gJ1jKl82wWVlnws4i5kqeUm1e-4rPqFjug0BcQKQLLHOiGhm0FcTBMs3orA0ZI7mOVOig6iOPN13Ova2ryiyW3jJycKXTZ6XCjb6rhzb2524dEvcRme-L1fxd9uInqxuS_KJdUiupDub0xWmgIHQcWk5o0E7pyhHiIZjSSdqAL3xeRA6tAAAA&shmds=v1_ATWGeePjnQgx9Du20_sxPP17bR_bDIAok5cW7PINbPZG_X6B4A&source=sh/x/job/li/m1/1#fpstate=tldetail&htivrt=jobs&htiq=Data+Analyst&htidocid=MFmZLfNtEdsNr_nSAAAAAA%3D%3D,WBD Careers - Warner Bros. Discovery
Digital Transformation Head-Engineering-BFSI- Bangalore,Spot Your Leaders & Consulting,الهند,technology jobs,https://www.google.com/search?ibp=htl;jobs&q=Technology+Jobs&htidocid=RZLUhdhJ4RJjlzdsAAAAAA%3D%3D&hl=ar-QA&shndl=37&shmd=H4sIAAAAAAAA_x3OvUpDQRCGYWxzCVZTWQjnHBVstIvxD-xiYxX2JONmZTNz2NlASgMh_uB1GCGIrd6Hx

# Conclusion

The ETL pipeline successfully ingested, processed, and analyzed software and IT job listings using SerpAPI, Apache Spark, and Databricks.

Key business insights derived from the Gold layer include:

- Roles with the most job openings: Data Analyst, Software Engineer, Backend Developer, and Full Stack Developer

- Top hiring companies: Accenture, Infosys, Cognizant, Oracle, and TCS

- Top job locations: Hyderabad, Bangalore, and pan-India/remote locations

Additionally, roles such as Data Analyst and Data Engineer show both high hiring volume and wide geographic distribution, indicating strong and sustained demand across multiple regions. Hyderabad emerged as a major hiring hub, particularly for analytics and engineering roles.

This project demonstrates how the Medallion Architecture enables clean, standardized, and analytics-ready datasets, empowering organizations to efficiently answer critical hiring, workforce planning, and job market intelligence questions.